# Quantium Data Analysis

In [2]:
#Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

%matplotlib inline

In [5]:
'''The merged data from task 1 can be used in task 2. However, when I check the data Quantium provided in task 2, the DATE
attritbute is incorrect for task 1. So I used the new data they provided in task 2'''
data = pd.read_csv('QVI_data.csv')
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LYLTY_CARD_NBR    264834 non-null  int64  
 1   DATE              264834 non-null  object 
 2   STORE_NBR         264834 non-null  int64  
 3   TXN_ID            264834 non-null  int64  
 4   PROD_NBR          264834 non-null  int64  
 5   PROD_NAME         264834 non-null  object 
 6   PROD_QTY          264834 non-null  int64  
 7   TOT_SALES         264834 non-null  float64
 8   PACK_SIZE         264834 non-null  int64  
 9   BRAND             264834 non-null  object 
 10  LIFESTAGE         264834 non-null  object 
 11  PREMIUM_CUSTOMER  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


In [15]:
#Change data type to save memory
int64 =  data.select_dtypes(include=['int64']).columns.tolist() 
data[int64] = data[int64].astype('int32')

data['TOT_SALES'] = data['TOT_SALES'].astype('float16') #Change float64 to float16

#Change DATE to datetime data
data['DATE'] = pd.to_datetime(data['DATE'],format = '%Y-%m-%d')

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   LYLTY_CARD_NBR    264834 non-null  int32         
 1   DATE              264834 non-null  datetime64[ns]
 2   STORE_NBR         264834 non-null  int32         
 3   TXN_ID            264834 non-null  int32         
 4   PROD_NBR          264834 non-null  int32         
 5   PROD_NAME         264834 non-null  object        
 6   PROD_QTY          264834 non-null  int32         
 7   TOT_SALES         264834 non-null  float16       
 8   PACK_SIZE         264834 non-null  int32         
 9   BRAND             264834 non-null  object        
 10  LIFESTAGE         264834 non-null  object        
 11  PREMIUM_CUSTOMER  264834 non-null  object        
dtypes: datetime64[ns](1), float16(1), int32(6), object(4)
memory usage: 16.7+ MB


The client has selected store numbers 77, 86 and 88 as trial stores and want 
control stores to be established stores that are operational for the entire 
observation period.
We would want to match trial stores to control stores that are similar to the trial
store prior to the trial period of Feb 2019 in terms of :
- Monthly overall sales revenue
- Monthly number of customers
- Monthly number of transactions per customer

Let's first create the metrics of interest and filter to stores that are present 
throughout the pre-trial period.

In [18]:
#Add a new month ID column in the data with format yyyymm
data['MONTH_ID'] = data['DATE'].dt.to_period('M')
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,MONTH_ID
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.000000,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,2018-10
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.699219,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,2018-09
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.599609,210,GRNWVES,YOUNG FAMILIES,Budget,2019-03
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.000000,175,NATURAL,YOUNG FAMILIES,Budget,2019-03
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.900391,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,2018-11


As we only considered stores that have sales every month over July 2018 to June 2019, stores that do not satisfy the condition need to be dropped

In [29]:
count_months = data.groupby('STORE_NBR')['MONTH_ID'].nunique()
count_months = count_months[count_months < 12]
print(count_months)

STORE_NBR
11      2
31      2
44     11
76      1
85      1
92      1
117    11
193     3
206     2
211     2
218    11
252     2
Name: MONTH_ID, dtype: int64


In [31]:
data = data[~data['STORE_NBR'].isin(count_months)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262251 entries, 2581 to 264833
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   LYLTY_CARD_NBR    262251 non-null  int32         
 1   DATE              262251 non-null  datetime64[ns]
 2   STORE_NBR         262251 non-null  int32         
 3   TXN_ID            262251 non-null  int32         
 4   PROD_NBR          262251 non-null  int32         
 5   PROD_NAME         262251 non-null  object        
 6   PROD_QTY          262251 non-null  int32         
 7   TOT_SALES         262251 non-null  float16       
 8   PACK_SIZE         262251 non-null  int32         
 9   BRAND             262251 non-null  object        
 10  LIFESTAGE         262251 non-null  object        
 11  PREMIUM_CUSTOMER  262251 non-null  object        
 12  MONTH_ID          262251 non-null  period[M]     
dtypes: datetime64[ns](1), float16(1), int32(6), object(4), p